In [1]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from sqlalchemy import create_engine,text
from google.api_core import retry
from langchain_community.agent_toolkits import create_sql_agent
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd
import tqdm as notebook_tqdm
import os

C:\Users\lcast\anaconda3\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load_ext dotenv
load_dotenv()
print(load_dotenv())

True


In [3]:
df=pd.read_csv("data/titanic.csv")
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Survived                 887 non-null    int64  
 1   Pclass                   887 non-null    int64  
 2   Name                     887 non-null    object 
 3   Sex                      887 non-null    object 
 4   Age                      887 non-null    float64
 5   Siblings/Spouses Aboard  887 non-null    int64  
 6   Parents/Children Aboard  887 non-null    int64  
 7   Fare                     887 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.6+ KB


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [6]:
db_path = "data/sqldb.db"
os.remove(db_path)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'data/sqldb.db'

In [7]:
engine=create_engine(f"sqlite:///data/sqldb.db")

def check_table_exists(engine, table_name):
    with engine.connect() as connection:
        query = text(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';")
        result = connection.execute(query).fetchone()
    return result is not None

table_name='titanic'

table_exists = check_table_exists(engine, table_name)

if table_exists:
    user_input = input("The table 'titanic' already exists. Do you want to replace it? (yes/no): ").strip().lower()

    if user_input == 'yes':
        # Replace the table if it already exists
        df.to_sql(table_name, engine, index=False, if_exists='replace')
        print("Table has been replaced.")
    else:
        print("No changes were made to the existing database.")
else:
    # Create the table if it doesn't exist
    df.to_sql("titanic", engine, index=False)
    print("Table 'titanic' has been created.")

The table 'titanic' already exists. Do you want to replace it? (yes/no):  yes


Table has been replaced.


In [8]:
#Connecting to database
db=SQLDatabase.from_uri(f"sqlite:///data/sqldb.db")

In [9]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['titanic']


"[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]"

#Create database following instructions in "Create sql database.md"


In [10]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=5,
    # other params...
)

In [14]:
chain=create_sql_query_chain(llm,db)
response = chain.invoke({"question": "How many  survivors?"})
print(response)

Question: How many survivors?
SQLQuery: SELECT COUNT("Survived") FROM titanic WHERE "Survived" = 1


In [16]:
query=response.split('SQLQuery:')[1].strip()
db.run(query)

'[(342,)]'

In [17]:
#Add QuerySQLDataBaseTool to the chain
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)
chain = write_query | execute_query

In [18]:
chain = write_query | execute_query

In [19]:
#Error because incorrect SQL format
chain.invoke({"question": "How many passengers are there?"})

'Error: (sqlite3.OperationalError) near "Question": syntax error\n[SQL: Question: How many passengers are there?\nSQLQuery: SELECT COUNT("Name") FROM titanic]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [ ]:
from langchain.chains.sql_database.prompt import SQL_PROMPTS

In [21]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [31]:
#Note: need to modify the prompt in order to get the correct query by removing the header 
#"SQL Query fenerated by the model" Even so, somettimes provides the wrong answer
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, create a syntactically correct query to run by not including "SQL Query:" string in the SQL query.
       Then execute the syntactically correct query and and answer the user question with the SQL result"
Use the following format:
     
Question: {question}
SQL Query: {query}
SQL Result: {result}
"""

)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many survivors"})

'Question: How many survivors\nSQL Query: SELECT COUNT("Survived") FROM titanic WHERE "Survived" = 1\nSQL Result: 500'

***Agents***

In [32]:
agent_executor = create_sql_agent(llm, db=db, verbose=True)

In [35]:
response=agent_executor.invoke(
    {
        "input": "Number of survivors by gender and number of children"
    }
)



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: titanicAction: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/Thought: I now have the schema of the titanic table. I can see that the columns "Survived", "Sex", and "Parents/Children Aboard" are relevant to the question. I should query the database to get the number of survivors by gender and number of

In [36]:
response['output']

'The number of survivors by gender and number of children is as follows: \n- Female survivors with 0 children: 153\n- Female survivors with 1 child: 46\n- Female survivors with 2 children: 30\n- Female survivors with 3 children: 3\n- Female survivors with 5 children: 1\n- Male survivors with 0 children: 80\n- Male survivors with 1 child: 19\n- Male survivors with 2 children: 10'